In [1]:
using JuMP, Ipopt

In [2]:
mod = Model(solver=IpoptSolver(print_level=0, max_iter=30000))
M = 500
@variable(mod, Δt >= 0, start = 1/M)
@NLexpression(mod, tf, Δt*M)
@variable(mod, -1000 <= x[0:M] <= 1000)
@variable(mod, 0 <= y[0:M] <= 1000)
@variable(mod, -200 <= vx[0:M] <= 200)
@variable(mod, -200 <= vy[0:M] <= 0)
@variable(mod, 0 <= θ[0:M] <= 2*pi)
@variable(mod, -4 <= ω[0:M] <= 4)
@variable(mod, 0 <= m[0:M] <= 10000)

@variable(mod, 0 <= u[0:M] <= 1)
@variable(mod, -1 <= utx[0:M] <= 1)
@variable(mod, -1 <= uty[0:M] <= 1)
@objective(mod, Max, m[M])

@constraint(mod, x[M] == 0)
@constraint(mod, y[M] == 0)
@constraint(mod, vx[M] == 0)
@constraint(mod, vy[M] == 0)
@constraint(mod, θ[M] == 0)
@constraint(mod, ω[M] ==0);

@constraint(mod, x[0] == 0)
@constraint(mod, y[0] == 1000)
@constraint(mod, vx[0] == 10)
@constraint(mod, vy[0] == -5)
@constraint(mod, θ[0] == 0)
@constraint(mod, ω[0] == 0)
@constraint(mod, m[0] == 10000)

m[0] = 10000

In [3]:
c1 = 5886000*0.3
c2 = 311*9.81
c3 = 300
g = 9.81;

In [4]:
@NLexpression(mod, dvx[i=0:M], c1*u[i]*utx[i]/m[i])
@NLexpression(mod, dvy[i=0:M], c1*u[i]*uty[i]/m[i])
@NLexpression(mod, dω[i=0:M], -c1*u[i]*(utx[i]*cos(θ[i]) - uty[i]*sin(θ[i]))/(c3*m[i]))
@NLexpression(mod, dm[i=0:M], -c1*u[i]/c2)

JuMP.JuMPArray{JuMP.NonlinearExpression,1,Tuple{UnitRange{Int64}}}(JuMP.NonlinearExpression[JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1505),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1506),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1507),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1508),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1509),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1510),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 variables
Solver is Ipopt,1511),JuMP.NonlinearExpression(Maximization problem with:
 * 13 linear constraints
 * 5011 vari

In [5]:
for j in 1:M
    @NLconstraint(mod, x[j] == x[j-1] + 0.5*Δt*(vx[j] + vx[j-1]))
    @NLconstraint(mod, y[j] == y[j-1] + 0.5*Δt*(vy[j] + vy[j-1]))
    @NLconstraint(mod, vx[j] == vx[j-1] + 0.5*Δt*(dvx[j] + dvx[j-1]))
    @NLconstraint(mod, vy[j] == vy[j-1] + 0.5*Δt*(dvy[j] + dvy[j-1]))
    @NLconstraint(mod, θ[j] == θ[j-1] + 0.5*Δt*(ω[j] + ω[j-1]))
    @NLconstraint(mod, ω[j] == ω[j-1] + 0.5*Δt*(dω[j] + dω[j-1]))
    @NLconstraint(mod, m[j] == m[j-1] + 0.5*Δt*(dm[j] + dm[j-1]))
end
mod

Maximization problem with:
 * 13 linear constraints
 * 3500 nonlinear constraints
 * 5011 variables
Solver is Ipopt

In [6]:
println("Solving...")
status = solve(mod)
println("Solver status: ", status)

Solving...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



Solver status: Error


In [7]:
using PyPlot
println(getValue(x)[0:M])

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in getValue(::JuMP.JuMPArray{JuMP.Variable,1,Tuple{UnitRange{Int64}}}, ::Vararg{JuMP.JuMPArray{JuMP.Variable,1,Tuple{UnitRange{Int64}}},N}) at ./deprecated.jl:30
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/cisprague/.julia/v0.5/IJulia/src/execute_request.jl:157
 in eventloop(::ZMQ.Socket) at /home/cisprague/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##13#19)() at ./task.jl:360
while loading In[7], in expression starting on line 2


[-5.92346e-43,-265.086,-116.33,177.646,-90.7494,30.5611,33.3094,-22.9096,11.9461,-63.5339,-6.25572,13.5228,22.3428,17.5215,0.765991,-6.46161,-7.08865,-5.85334,-1.77758,0.846165,1.62545,2.00436,1.6042,0.849083,0.33433,0.316941,0.431157,0.377873,0.231935,0.305699,0.450941,0.45565,0.400678,0.495656,0.60126,0.581173,0.529682,0.564813,0.54723,0.377801,0.177703,0.0608604,-0.040441,-0.188,-0.313079,-0.359916,-0.371009,-0.381403,-0.379516,-0.358908,-0.330799,-0.297808,-0.259099,-0.216878,-0.170351,-0.119464,-0.0746628,-0.0474484,-0.0445331,-0.0995077,-0.242125,-0.360287,-0.227269,0.128789,0.284531,0.0316521,-0.227475,-0.149598,0.023688,0.000708902,-0.0463176,0.142449,0.432777,0.521039,0.354653,0.147438,0.108283,0.159075,0.0402453,-0.18442,-0.0757774,0.423957,0.68309,0.335768,-0.0850251,-0.0209538,0.292727,0.370466,0.222893,0.166499,0.297912,0.437561,0.414532,0.243062,0.0692015,-0.00717156,-0.00775265,0.0246313,0.101031,0.238408,0.385695,0.450736,0.382781,0.211234,0.0398876,-0.0249607,0.0539237